In [ ]:
import requests
from bs4 import BeautifulSoup


In [ ]:
# 이미지 저장 디렉터리 생성
import os
os.makedirs('downloaded_images', exist_ok=True)


In [ ]:
# 위키미디어 공용의 특정 카테고리 URL
img_url = 'https://commons.wikimedia.org/wiki/Category:Mountains'
# img_url = 'https://commons.wikimedia.org/wiki/Category:Bicycles'


In [ ]:
response = requests.get(img_url)
soup = BeautifulSoup(response.text, 'html.parser')


In [ ]:
########################################################################
### Crawling & Labelling
### 1. 폴더 구분

In [ ]:
#### 크롤링을 통한 이미지 수집
# 다운로드된 이미지 수
downloaded_images = 0

for item in soup.find_all('div', class_='gallerytext'):
    if downloaded_images >= 10:  # 다운로드 이미지 갯수한정
        break

    a_tag = item.find('a')
    if a_tag:
        img_page_url = 'https://commons.wikimedia.org' + a_tag.get('href')

        img_page_response = requests.get(img_page_url)
        img_page_soup = BeautifulSoup(img_page_response.text, 'html.parser')
        img_tag = img_page_soup.find('div', class_='fullImageLink').find('a')
        img_url = img_tag.get('href')
        print(img_url)
        # 파일 확장자 검사
        if img_url.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            try:
                headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
                  # (일부)웹 서버에서 User-Agent 헤더를 검사, 브라우저가 아닌 클라이언트의 요청 차단
                  # requests.get() 함수에 웹 브라우저의 User-Agent 값을 포함하는 헤더를 추가하여 다운로드 시도
                img_data = requests.get(img_url, headers=headers).content

                # Colab 루트 디렉토리에 'downloaded_images' 폴더를 생성하고 이미지 저장
                # filename = os.path.join('/content/downloaded_images', img_url.split('/')[-1])
                filename = '/content/m/'+img_url.split('/')[-1]

                with open(filename, 'wb') as f:
                    f.write(img_data)
                print(f'이미지가 저장되었습니다: {filename}')
                downloaded_images += 1
            except Exception as e:
                print(f"이미지 다운로드 중 오류 발생: {e}")
        else:
            print(f'지원되지 않는 파일 형식: {img_url}')

https://upload.wikimedia.org/wikipedia/commons/b/b9/1_A_weeny_window.jpg
이미지가 저장되었습니다: /content/m/1_A_weeny_window.jpg


In [ ]:
########################################################################
### Crawling & Labelling
### 2. 파일명에 라벨 포함

# filename = f'{category_prefix}_{downloaded_images+1}{file_extension}'

In [ ]:
# 이미지 저장 함수
def download_images1(url, category_prefix):
    # 이미지를 저장할 디렉터리 생성
    # os.makedirs('downloaded_images/s', exist_ok=True)

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 다운로드된 이미지 수
    downloaded_images = 0

    for item in soup.find_all('div', class_='gallerytext'):
        if downloaded_images >= 5:
            break

        a_tag = item.find('a')
        if a_tag:
            img_page_url = 'https://commons.wikimedia.org' + a_tag.get('href')

            img_page_response = requests.get(img_page_url)
            img_page_soup = BeautifulSoup(img_page_response.text, 'html.parser')
            img_tag = img_page_soup.find('div', class_='fullImageLink').find('a')
            img_url = img_tag.get('href')
            # print(img_url)

            # 파일 확장자 검사 및 파일 이름 생성
            # file_extension = os.path.splitext(img_url)[1]
            file_extension = img_url.split('.')[-1]
            # print(file_extension.lower())

            filename = f'{category_prefix}_{downloaded_images+1}{file_extension}'
            # filepath = os.path.join('downloaded_images/s', filename)
            filepath = filename

            if file_extension.lower() in ('png', 'jpg', 'jpeg', 'gif'):
                try:
                    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
                    img_data = requests.get(img_url, headers=headers).content
                    with open(filepath, 'wb') as f:
                        f.write(img_data)
                    print(f'이미지가 저장되었습니다: {filepath}')
                    downloaded_images += 1
                except Exception as e:
                    print(f"이미지 다운로드 중 오류 발생: {e}")
            else:
                print(f'지원되지 않는 파일 형식: {img_url}')

In [ ]:
url_m = 'https://commons.wikimedia.org/wiki/Category:Mountains'
url_b = 'https://commons.wikimedia.org/wiki/Category:Automobiles'

download_images1(url_m, 'm')  # 산 이미지에 'm' 접두사 추가
download_images1(url_b, 'b')  # 자전거 이미지에 'b' 접두사 추가

In [ ]:
########################################################################
### Crawling & Labelling
### 3. 라벨 파일을 별도로 작업 -> index로 매칭

# filename = f'{category_prefix}_{downloaded_images+1}{file_extension}'

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import csv


In [ ]:
# 이미지 저장 함수
def download_images3(url, category_prefix):

    # 클래스 정보가 포함된 CSV 파일 생성
    with open('image_labels.b.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['image_name', 'class'])  # CSV 파일 헤더

        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 다운로드된 이미지 수
        downloaded_images = 0

        for item in soup.find_all('div', class_='gallerytext'):
            if downloaded_images >= 10:
                break

            a_tag = item.find('a')
            if a_tag:
                img_page_url = 'https://commons.wikimedia.org' + a_tag.get('href')
                img_page_response = requests.get(img_page_url)
                img_page_soup = BeautifulSoup(img_page_response.text, 'html.parser')
                img_tag = img_page_soup.find('div', class_='fullImageLink').find('a')
                img_url = img_tag.get('href')

                # 파일 확장자 검사 및 파일 이름 생성
                file_extension = img_url.split('.')[-1]
                filename = f'{category_prefix}_{downloaded_images+1}{file_extension}'
                # filepath = '/content/aaa/'+filename
                filepath = filename

                # HEAD 요청을 보내어 파일 사이즈 확인
                head_response = requests.head(img_url)
                size_bytes = int(head_response.headers.get('content-length', 0))

                # 파일 사이즈가 max_size_bytes 이하인 경우에만 다운로드
                max_size_bytes = 100000
                if size_bytes <= max_size_bytes:
                    if file_extension.lower() in ('jpg'): # ('.png', '.jpg', '.jpeg', '.gif'):
                        try:
                            headers = {
                                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
                            }
                            print(size_bytes)
                            img_data = requests.get(img_url, headers=headers).content
                            with open(filepath, 'wb') as f:
                                f.write(img_data)
                            print(f'이미지가 저장되었습니다: {filepath}')
                            downloaded_images += 1

                            # 이미지 이름과 클래스 정보를 CSV 파일에 기록
                            writer.writerow([filename, category_prefix])
                        except Exception as e:
                            print(f"이미지 다운로드 중 오류 발생: {e}")
                    else:
                        print(f'지원되지 않는 파일 형식: {img_url}')
                else:
                    print(f'파일이 너무 큽니다 (사이즈: {size_bytes} 바이트)')

In [ ]:
url_b = 'https://commons.wikimedia.org/wiki/Category:Bicycles'
download_images3(url_b, 'b')  # 자전거 이미지에 'b' 접두사 추가


1905
이미지가 저장되었습니다: b_1jpg
1905
이미지가 저장되었습니다: b_2jpg
1905
이미지가 저장되었습니다: b_3jpg
1905
이미지가 저장되었습니다: b_4jpg
1905
이미지가 저장되었습니다: b_5jpg
1905
이미지가 저장되었습니다: b_6jpg
1905
이미지가 저장되었습니다: b_7jpg
1905
이미지가 저장되었습니다: b_8jpg
1905
이미지가 저장되었습니다: b_9jpg
1905
이미지가 저장되었습니다: b_10jpg
